# Quét span vectors và tạo report JSON (`span_vectors_full_info.json`)

## Mục tiêu của script
Script này dùng để:
- **Quét đệ quy** toàn bộ thư mục chứa vector span (`sentence_*.pt`)
- **Phân tích tên file + cấu trúc thư mục** để lấy thông tin định danh:
  - `verb_name` (ví dụ: `begin_2`)
  - `sentence_idx` (ví dụ: `0`)
  - `arg_label` (ví dụ: `ARG-0`)
- **Tự xác định file JSON dataset gốc** tương ứng (Train/Test, GramVar/ParaVE)
- **Tra cứu nội dung text**:
  - `full_sentence`: câu đầy đủ từ dataset
  - `argument_text`: span text ứng với nhãn `ARG-x`
- **Xuất report JSON** chứa đầy đủ metadata + đường dẫn vector `.pt`

## Thư viện sử dụng
- `json`: đọc/ghi dữ liệu JSON.
- `os`: xử lý đường dẫn, tách thư mục/tên file, tạo path chuẩn.
- `glob`: tìm file theo pattern, hỗ trợ quét đệ quy với `**`.
- `tqdm.auto`: hiển thị progress bar khi xử lý nhiều file.

## Cấu hình đường dẫn & file đầu ra

### Các biến cấu hình chính
- `BASE_VECTOR_SEARCH_DIR`  
  Thư mục gốc chứa các folder vector span. Script sẽ **quét đệ quy** từ đây để tìm `sentence_*.pt`.

- `BASE_DATASET_DIR`  
  Thư mục gốc chứa dataset JSON (dùng để tra cứu text theo `verb_name` + split).

- `OUTPUT_JSON_FILE`  
  Tên file JSON kết quả đầu ra (report tổng hợp).

> Lưu ý: nếu chạy trên Google Colab, cần mount Google Drive để các đường dẫn `/content/drive/...` tồn tại.

## Hàm `get_json_path(verb_name, parent_dir_name)`

### Mục đích
Xác định **đường dẫn file dataset JSON** cần load dựa trên tên thư mục cha (`parent_dir_name`) và tên verb (`verb_name`).

### Logic chính
- Nếu `parent_dir_name` chứa `"test"` → split = **Test**, ngược lại **Train**
- Nếu `parent_dir_name` chứa `"gramvar"` → dataset = **Split_GramVar**, ngược lại **Split_ParaVE**
- Tên file JSON được tạo theo quy tắc:
  - Test: `{verb_name}_test_set.json`
  - Train: `{verb_name}_train_set.json`

### Hàm trả về
- `json_path`: đường dẫn đầy đủ tới file JSON dataset
- `dataset_type`: `Split_GramVar` hoặc `Split_ParaVE`
- `split_type`: `Train` hoặc `Test`


## Hàm chính `process_all_vectors_to_json()`

### Bước 1 — Tìm tất cả file vector `.pt`
- Tạo pattern: `.../**/sentence_*.pt`
- Dùng `glob.glob(..., recursive=True)` để lấy toàn bộ file phù hợp
- Nếu không tìm thấy file nào → in cảnh báo và dừng

### Bước 2 — Tạo cache dataset JSON để tăng tốc
- `dataset_cache = {}` lưu nội dung JSON theo `json_path`
- Mục tiêu: **mỗi file dataset chỉ load 1 lần**, tránh đọc đi đọc lại khi có nhiều vector thuộc cùng verb

### Bước 3 — Duyệt từng file `.pt` và parse thông tin định danh
Với mỗi `pt_path`:
- Tách đường dẫn theo `os.sep`
- Quy ước cấu trúc:
  - `filename = parts[-1]` → ví dụ `sentence_0_ARG_0.pt`
  - `verb_name = parts[-2]` → ví dụ `begin_2`
  - `parent_dir = parts[-3]` → tên folder ngay phía trên `verb_name`

Chỉ xử lý nếu `parent_dir` chứa `"span_adaptation_vectors"` (lọc đúng nhóm thư mục mong muốn).

### Bước 4 — Lấy `sentence_idx` từ tên file
- Tên file dạng: `sentence_<idx>_...`
- Lấy `sent_idx_str = filename.split('_')[1]`
- Nếu không phải số → bỏ qua
- `sent_idx = int(sent_idx_str)`

### Bước 5 — Lấy nhãn argument `ARG-x`
- Ví dụ: `sentence_0_ARG_0.pt`
- Bỏ `.pt` → `sentence_0_ARG_0`
- `arg_part = base_name.split('_', 2)[2]` → `ARG_0`
- Đổi `_` thành `-` → `ARG-0`

### Bước 6 — Xác định file dataset JSON tương ứng
- Gọi: `json_path, ds_type, split_type = get_json_path(verb_name, parent_dir)`
- Nếu `json_path` chưa có trong cache:
  - Nếu tồn tại file → load JSON vào cache
  - Nếu không tồn tại → cache `None` (để lần sau khỏi kiểm tra lại)

### Bước 7 — Tra cứu text từ dataset
- Nếu dataset tồn tại và `sent_idx < len(data)`:
  - `entry = data[sent_idx]`
  - `full_sentence = entry.get("text", "")`
  - `args = entry.get("arguments", {})`
  - `argument_text = args.get(arg_label, None)`

Chỉ khi `argument_text` tồn tại mới ghi nhận kết quả.

### Bước 8 — Tạo object kết quả và lưu vào danh sách
Mỗi item hợp lệ sẽ có dạng:
- `corpus_id`: `verb_name`
- `sentence_idx`: chỉ số câu
- `span_id`: nhãn span (ARG-x)
- `argument_text`: nội dung span
- `full_sentence`: câu đầy đủ
- `dataset_source`: ví dụ `Split_GramVar/Train`
- `vector_file_path`: đường dẫn file `.pt`
- `file_name`: tên file vector

### Bước 9 — Xuất report JSON
- Ghi `final_results` ra `OUTPUT_JSON_FILE`
- `ensure_ascii=False` để giữ tiếng Việt
- In số lượng mục xuất thành công và đường dẫn file kết quả

## Kết quả đầu ra
File `span_vectors_full_info.json` là một danh sách các object, mỗi object ứng với **một span vector** đã tra được text từ dataset, phục vụ:
- kiểm tra dữ liệu
- đối chiếu vector ↔ câu gốc ↔ argument span
- thống kê / phân tích / debug pipeline

In [ ]:
import json
import os
import glob
from tqdm.auto import tqdm

print("--- BẮT ĐẦU QUÉT VÀ TẠO REPORT JSON ---")

# --- 1. CẤU HÌNH ---
# Thư mục gốc chứa các folder vector (Script sẽ quét đệ quy từ đây)
BASE_VECTOR_SEARCH_DIR = "/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_gramvar_inner_content"

# Thư mục chứa Dataset gốc (để tra text)
BASE_DATASET_DIR = "/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Train"

# File kết quả đầu ra (Dạng JSON)
OUTPUT_JSON_FILE = "span_vectors_full_info.json"

def get_json_path(verb_name, parent_dir_name):
    """Xác định đường dẫn file JSON dataset dựa trên tên thư mục"""
    is_test = "test" in parent_dir_name.lower()
    is_gramvar = "gramvar" in parent_dir_name.lower()

    dataset_type = "Split_GramVar" if is_gramvar else "Split_ParaVE"
    split_type = "Test" if is_test else "Train"

    suffix = "_test_set.json" if is_test else "_train_set.json"
    json_filename = f"{verb_name}{suffix}"

    return os.path.join(BASE_DATASET_DIR, dataset_type, split_type, json_filename), dataset_type, split_type

def process_all_vectors_to_json():
    # Tìm tất cả file .pt trong các thư mục con
    search_pattern = os.path.join(BASE_VECTOR_SEARCH_DIR, "**", "sentence_*.pt")
    all_pt_files = glob.glob(search_pattern, recursive=True)

    print(f"-> Tìm thấy tổng cộng {len(all_pt_files)} file vector.")

    if len(all_pt_files) == 0:
        print("⚠️ Không tìm thấy file nào.")
        return

    # Cache dữ liệu JSON Dataset để tối ưu tốc độ
    dataset_cache = {}

    # Danh sách kết quả để lưu vào JSON
    final_results = []

    success_count = 0

    for pt_path in tqdm(all_pt_files, desc="Đang xử lý"):
        try:
            # Phân tích đường dẫn file vector
            parts = pt_path.split(os.sep)
            if len(parts) < 3: continue

            filename = parts[-1]          # VD: sentence_0_ARG_0.pt
            verb_name = parts[-2]         # VD: begin_2
            parent_dir = parts[-3]        # VD: span_adaptation_vectors_...

            # Chỉ xử lý các thư mục liên quan đến span vectors
            if "span_adaptation_vectors" not in parent_dir: continue

            # Lấy Sentence Index
            sent_idx_str = filename.split('_')[1]
            if not sent_idx_str.isdigit(): continue
            sent_idx = int(sent_idx_str)

            # Lấy ARG label (xử lý tên file: sentence_0_ARG_0.pt -> ARG-0)
            base_name = filename.replace('.pt', '')
            arg_part = base_name.split('_', 2)[2]
            arg_label = arg_part.replace('_', '-')

            # Xác định file Dataset gốc
            json_path, ds_type, split_type = get_json_path(verb_name, parent_dir)

            # Load Dataset JSON (sử dụng Cache)
            if json_path not in dataset_cache:
                if os.path.exists(json_path):
                    with open(json_path, 'r', encoding='utf-8') as f:
                        dataset_cache[json_path] = json.load(f)
                else:
                    dataset_cache[json_path] = None

            data = dataset_cache[json_path]

            # Tra cứu Text
            target_text = None
            full_sentence = None

            if data and sent_idx < len(data):
                entry = data[sent_idx]
                full_sentence = entry.get('text', '')
                args = entry.get('arguments', {})
                target_text = args.get(arg_label, None)

            if target_text:
                # Tạo Object thông tin
                item_info = {
                    "corpus_id": verb_name,
                    "sentence_idx": sent_idx,
                    "span_id": arg_label,
                    "argument_text": target_text,
                    "full_sentence": full_sentence,
                    "dataset_source": f"{ds_type}/{split_type}",
                    "vector_file_path": pt_path,
                    "file_name": filename
                }
                final_results.append(item_info)
                success_count += 1

        except Exception as e:
            continue

    # Lưu danh sách ra file JSON
    with open(OUTPUT_JSON_FILE, "w", encoding="utf-8") as f:
        json.dump(final_results, f, indent=4, ensure_ascii=False)

    print(f"\n✅ HOÀN TẤT! Đã xuất {success_count} mục ra file JSON.")
    print(f"📁 File kết quả: {os.path.abspath(OUTPUT_JSON_FILE)}")

if __name__ == "__main__":
    process_all_vectors_to_json()

--- BẮT ĐẦU QUÉT VÀ TẠO REPORT JSON ---
-> Tìm thấy tổng cộng 10390 file vector.


Đang xử lý:   0%|          | 0/10390 [00:00<?, ?it/s]


✅ HOÀN TẤT! Đã xuất 0 mục ra file JSON.
📁 File kết quả: /content/span_vectors_full_info.json
